In [1]:
%%init_spark
launcher.master = "local[*]"
launcher.conf.spark.driver.memory= "30G"
launcher.conf.set("spark.hadoop.fs.s3a.connection.maximum", "8192")
launcher.conf.set("spark.hadoop.fs.s3a.threads.max", "2048")

In [2]:
import java.time.LocalDate
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{SparkSession, Row, DataFrame, Dataset}
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.expressions.UserDefinedFunction
import spark.implicits._
import org.apache.spark.sql.types._

Intitializing Scala interpreter ...

Spark Web UI available at http://localhost:4042
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1598369227341)
SparkSession available as 'spark'


import java.time.LocalDate
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{SparkSession, Row, DataFrame, Dataset}
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.expressions.UserDefinedFunction
import spark.implicits._
import org.apache.spark.sql.types._


In [34]:
val model = "ABTesting"  //"noisy_clicks"
val env = "stg"
val date = "2020-08-21"

model: String = ABTesting
env: String = stg
date: String = 2020-08-21


In [35]:
var modelPredictionsPath = s"s3a://midgar-aws-workspace/${env}/shinra/modelling/$model/weighted_trendfiltering_banner_events/user_categories/model_date=$date"
val categoryScoreColumnName = "category_trend_scores"
val groundTruthPath = s"s3a://midgar-aws-workspace/prod/shinra/telescope/base/l1_banner_events_base_table"
val topK = 4

modelPredictionsPath: String = s3a://midgar-aws-workspace/stg/shinra/modelling/ABTesting/weighted_trendfiltering_banner_events/user_categories/model_date=2020-08-21
categoryScoreColumnName: String = category_trend_scores
groundTruthPath: String = s3a://midgar-aws-workspace/prod/shinra/telescope/base/l1_banner_events_base_table
topK: Int = 4


In [36]:
spark.read.parquet(modelPredictionsPath).select(size($"category_trend_scores")).show(1, false)

+---------------------------+
|size(category_trend_scores)|
+---------------------------+
|64                         |
+---------------------------+
only showing top 1 row



In [9]:
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{SparkSession, Row, DataFrame, Dataset}

case class CustomerAffinities(customer_id: Long, category_scores: Seq[(String, Double)])
def getTopUserCategories(
  userAffinities: Dataset[CustomerAffinities],
  topK: Int,
  filterIds:Seq[String]
): RDD[(Long, Array[String])] = {
  import userAffinities.sparkSession.implicits._
  userAffinities.map {
    case CustomerAffinities(customerId: Long, probArray: Seq[(String, Double)]) =>
      (customerId, probArray.toArray.sortBy(-_._2).take(topK).map(_._1))
  }.rdd
}


import org.apache.spark.rdd.RDD
import org.apache.spark.sql.{SparkSession, Row, DataFrame, Dataset}
defined class CustomerAffinities
getTopUserCategories: (userAffinities: org.apache.spark.sql.Dataset[CustomerAffinities], topK: Int, filterIds: Seq[String])org.apache.spark.rdd.RDD[(Long, Array[String])]


In [10]:
def getUserSales(spark:SparkSession, startDateStr: String,endDateStr: String): DataFrame={
    spark.read.parquet(groundTruthPath)
    .filter($"dt".between(startDateStr,endDateStr))
    .filter($"sales".isNotNull &&  $"sales" > 0)
    .select($"customer_id".cast(LongType), $"cma_id", $"sales", $"dt".cast(StringType))
}

getUserSales: (spark: org.apache.spark.sql.SparkSession, startDateStr: String, endDateStr: String)org.apache.spark.sql.DataFrame


In [15]:
/*
def getTopSalesCategories(sales: DataFrame): RDD[(Long, Array[String])] = {
  import org.apache.spark.mllib.rdd.MLPairRDDFunctions.fromPairRDD
  import sales.sparkSession.implicits._
    
  sales.rdd.
    map {
      case Row(customerId: Long, cma_id: String, sales: Long, dt: String) =>
        (customerId, (cma_id, sales, dt))
    }
    .topByKey(1)(Ordering.by[(String, Long, String), (Long, String)](a =>
      (a._2, a._3))(Ordering.fromLessThan[(Long, String)]((x, y) =>
      if (y._1 != x._1) x._1 < y._1 else y._2 < x._2))).
    map {
      case (customerId: Long, recentCategories: Array[(String, Long, String)]) =>
        (customerId, recentCategories.map(_._1))
    }
}*/


def getTopSalesCategories(sales: DataFrame): RDD[(Long, Array[String])] = {
  import org.apache.spark.mllib.rdd.MLPairRDDFunctions.fromPairRDD
  import sales.sparkSession.implicits._
   
    
    val modelCategories = Seq("9", "10", "12", "14", "15", "16", "17", "18", "19",
    "20", "22", "23", "24", "25", "26", "29",
    "32", "33", "34", "35", "36", "37", "38", "39",
    "40", "41", "42", "43", "44", "45", "46", "48", "49",
    "50", "51", "52", "53", "55", "56", "57", "58", "59",
    "60", "352", "391", "357", "358",
    "420", "421", "423", "426", "429", "437", "442", "443", "444", "445", "446",
    "6803", "6797", "6808", "6809", "6810", "6811")
    
  val nsales = sales.filter($"cma_id".isin(modelCategories:_*))  
    
  nsales.rdd.
    map {
      case Row(customerId: Long, cma_id: String, sales: Long, dt: String) =>
        (customerId, (cma_id, sales, dt))
    }
    .groupByKey()
    .map {
      case (customerId: Long, recentCategories: Iterable[(String, Long, String)]) =>
        (customerId, recentCategories.toArray.map(_._1))
    }
}

getTopSalesCategories: (sales: org.apache.spark.sql.DataFrame)org.apache.spark.rdd.RDD[(Long, Array[String])]


In [31]:
import scala.reflect.ClassTag
import org.apache.spark.mllib.evaluation.{MultilabelMetrics, RankingMetrics}
import org.apache.spark.rdd.RDD
class FixedRankingMetrics[T: ClassTag](predictionAndLabels: RDD[(Array[T], Array[T])]) extends RankingMetrics(predictionAndLabels: RDD[(Array[T], Array[T])]) {

  override def precisionAt(k: Int): Double = {
    require(k > 0, "ranking position k should be positive")
    predictionAndLabels.map {
      case (pred, lab) =>
        val predSetSize = pred.toSet.size
        val labSet = lab.toSet
        val labSetSize = labSet.size
        if (predSetSize != pred.length) {
          logError("Duplicates in predictions, check prediction data")
          throw new IllegalArgumentException("Duplicates in predictions, check prediction data")
        } else if (labSet.nonEmpty) {
          val n = math.min(pred.length, k)
          var i = 0
          var cnt = 0
          while (i < n) {
            if (labSet.contains(pred(i))) {
              cnt += 1
            }
            i += 1
          }
          cnt.toDouble / math.min(predSetSize, k)
        } else {
          logWarning("Empty ground truth set, check input data")
          0.0
        }
    }.mean()
  }
}

import scala.reflect.ClassTag
import org.apache.spark.mllib.evaluation.{MultilabelMetrics, RankingMetrics}
import org.apache.spark.rdd.RDD
defined class FixedRankingMetrics


In [16]:
  def getRankingMetricsAtK(
    spark: SparkSession,
    userTopCategories: RDD[(Long, Array[String])],
    userTopSalesCategories: RDD[(Long, Array[String])],
    Ks: Array[Int],
    registerToDataDog: Boolean = true,
    modelName: String = "not-specified",
    t1: Option[Long] = None
  ): DataFrame = {
    import spark.sqlContext.implicits._

    println(s"userTopCategories: ${userTopCategories.count}")
    println(s"userTopSalesCategories: ${userTopSalesCategories.count}")

    val relevantDocuments = userTopCategories.join(userTopSalesCategories).map(_._2)
    println(s"relevantDocuments: ${relevantDocuments.count}")

    val metrics = new FixedRankingMetrics(relevantDocuments)
    spark.sparkContext.parallelize(Ks.map {
      k =>
        val precision = metrics.precisionAt(k)
        val ndcg = metrics.ndcgAt(k)
        val meanAP = metrics.meanAveragePrecision
        (k, precision, ndcg, meanAP)
    }).toDF("k", "precision_at_k", "ndcg_at_k", "mean_avg_precison")
  }

getRankingMetricsAtK: (spark: org.apache.spark.sql.SparkSession, userTopCategories: org.apache.spark.rdd.RDD[(Long, Array[String])], userTopSalesCategories: org.apache.spark.rdd.RDD[(Long, Array[String])], Ks: Array[Int], registerToDataDog: Boolean, modelName: String, t1: Option[Long])org.apache.spark.sql.DataFrame


In [37]:
val modelPredictions = spark.read.parquet(modelPredictionsPath)
  .select($"customer_id", $"category_trend_scores" as "category_scores")
  .as[CustomerAffinities]

modelPredictions: org.apache.spark.sql.Dataset[CustomerAffinities] = [customer_id: bigint, category_scores: array<struct<category_id:string,probability:double>>]


In [38]:
val topPredictionsPerUser = getTopUserCategories(modelPredictions, topK, Seq())


topPredictionsPerUser: org.apache.spark.rdd.RDD[(Long, Array[String])] = MapPartitionsRDD[149] at rdd at <console>:27


In [39]:
val salesCategories = getUserSales(spark, "2020-08-22", "2020-08-22")
salesCategories.count()

salesCategories: org.apache.spark.sql.DataFrame = [customer_id: bigint, cma_id: string ... 2 more fields]
res10: Long = 9247304


In [40]:
val topSalesPerUser = getTopSalesCategories(salesCategories)
topSalesPerUser.count()

topSalesPerUser: org.apache.spark.rdd.RDD[(Long, Array[String])] = MapPartitionsRDD[167] at map at <console>:84
res11: Long = 5447855


In [41]:
val precisionAtK_Sales = getRankingMetricsAtK(
    spark,
    topPredictionsPerUser,
    topSalesPerUser,
    (1 to topK).toArray,
    registerToDataDog = false,
    "")

userTopCategories: 101793436
userTopSalesCategories: 5447855
relevantDocuments: 5397946


precisionAtK_Sales: org.apache.spark.sql.DataFrame = [k: int, precision_at_k: double ... 2 more fields]


In [42]:
precisionAtK_Sales.show(20,false)

+---+-------------------+-------------------+-------------------+
|k  |precision_at_k     |ndcg_at_k          |mean_avg_precison  |
+---+-------------------+-------------------+-------------------+
|1  |0.3464841997307863 |0.3464841997307861 |0.44529838203992556|
|2  |0.46412996721345473|0.41928093589809523|0.44529838203992556|
|3  |0.5671356289966589 |0.4731293462310769 |0.44529838203992556|
|4  |0.6449846694773652 |0.5092145561818261 |0.44529838203992556|
+---+-------------------+-------------------+-------------------+

